## Can trading using RSI beat buying and holding a stock?

### Background: 
The Relative Strength Index (RSI) is a momentum indicator that describes the current price relative to average high and low prices over a previous trading period. This indicator estimates overbought or oversold status and helps spot trend reversals, price pullbacks, and the emergence of bullish or bearish markets.

The indicator was originally developed by J. Welles Wilder Jr. and introduced in his seminal 1978 book, “New Concepts in Technical Trading Systems.”

An asset is usually considered overbought when the RSI is above 70% and oversold when it is below 30%.

Here is the process of calculating RSI:

1. For 14 periods, calculate the difference in price from the current period and the previous period;
2. For each period, record each positive change in price as a gain and each negative change as a loss;
3. On the 14th period, calculate the arithmetic mean of the gains and losses for the entire 14 day period (e.g. gains / 14 & losses / 14);
4. Use these values to calculate the RS
5. Use the RS value to calculate the RSI
6. For each proceeding period, use only the previous RSI value to calculate the next average value by multiplying by our lookback period – 1 (e.g. 13 for a lookback of 14).
7. Add the value obtained in step 6 to the current day’s value (do this for both the gains and losses)

<img src="rsi.jpeg" width=300 height=300 />

### Resource:

- https://www.alpharithms.com/relative-strength-index-rsi-in-python-470209
- https://www.tradingview.com/support/solutions/43000502338-relative-strength-index-rsi/
- https://en.wikipedia.org/wiki/Relative_strength_index

### Import Libraries and Finance Data 

In [827]:
# import basic libraries
import pandas as pd
import pandas_ta as ta
import numpy as np
import altair as alt
from IPython.display import display_html
from itertools import chain,cycle

# adjust the width of jupyter notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

#three ways to import finance data, we will mainly use yfinance 
#https://pypi.org/project/yfinance    OR  https://www.alpharithms.com/python-financial-data-491110/

# import quandl
# df = quandl.get('WIKI/NVDA')

# import pandas_datareader as pdr
# #request API call from https://data.nasdaq.com/sign-up
# df = pdr.get_data_quandl("NVDA", api_key="vjsci8v8EfV-BpoQmxbR")

import yfinance as yf

C:\Users\nick\AppData\Local\Temp\ipykernel_35216\1710285548.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [828]:
## Function to show dataframes side by side
## https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
def display(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [829]:
# Get finance data, valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
Ticker="tru"
df = yf.Ticker(Ticker).history(period='3y')
df=df.reset_index()
df['Date']=pd.to_datetime(df['Date'])
df=df[['Date','Open','High','Low','Close']]
df['Close']=round(df['Close'],2)

In [830]:
df.head()

,Date,Open,High,Low,Close
0,2019-04-08,67.390566,67.608203,66.737629,67.45
1,2019-04-09,67.321313,67.420241,66.826667,67.07
2,2019-04-10,67.311426,67.964354,67.153135,67.65
3,2019-04-11,67.914896,67.929735,67.420250,67.72
4,2019-04-12,67.875330,69.497768,67.657685,69.36


### Calculating RSI Value

My attempt on calculating the RSI value using simple average. But a typical RSI value is calculating using Wilder’s original smoothing method
<img src="wsm.jpeg" width=300 height=300 />

In [831]:
#using the ta library, RSI values are calculated using daily close price. The default RSI period is 14 days.
df['RSI']=round(ta.rsi(df['Close'], timeperiod=14),2)

#Dropping the first 14 days without 14 days and reset index
df.drop(range(0,14),inplace=True)
df.reset_index(drop=True,inplace=True)

# Value is considered "overbought" when it exceeds 70 and "oversold" when indictator is below 30
df['Overbought']=70
df['Oversold']=30

In [832]:
# uncomment if you want to see how RSI is calculated using Simple Averages

# df['Difference'] = df['Close'].diff()
# df['Difference'].fillna(0,inplace=True)
# df['PercentChange'] = round(df['Difference']/ df['Close']*100,2)
# df.loc[df['Difference']>0, 'Gain'] = df['Difference']
# df.loc[df['Difference']<0, 'Loss'] = -df['Difference']
# df['Gain'].fillna(0,inplace=True)
# df['Loss'].fillna(0,inplace=True)
# for i in range(0,df.shape[0]-14):
#     sumGain=0
#     sumLoss=0
#     for index, row in df[1+i:14+i].iterrows():
#         sumGain=sumGain+row['Gain']
#         sumLoss=sumLoss+row['Loss']
#     df.loc[df.index==(14+i),'AvgGain']=round(sumGain/8,2)
#     df.loc[df.index==(14+i),'SumGain']=sumGain    
#     df.loc[df.index==(14+i),'AvgLoss']=round(sumLoss/8,2)    
#     df.loc[df.index==(14+i),'SumLoss']=sumLoss    
# df['RS'] = round(df['AvgGain']/df['AvgLoss'],2)
# df['AvgGain'].fillna(0,inplace=True)
# df['AvgLoss'].fillna(0,inplace=True)
# df['RS'].fillna(0,inplace=True)
# df['RSI'] = round(100-(100/(1+df['RS'])),2)
# df=df.reset_index()
# df.drop(range(0,14),inplace=True)

In [833]:
#Prepare of Stock graph
Stock = alt.Chart(df).mark_line(point=alt.OverlayMarkDef(color="grey",size=15)).encode(
    x='Date:T',
    y='Close:Q',
    tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('Close:Q', format=',.2f')]
).properties(
    title='Stock Price',
    width=800,
    height=300
)


In [834]:
#Plotting RSI chart with Oversold and Overbought lines
RSI = alt.Chart(df).mark_line().encode(
    x='Date:T',
    y='RSI:Q',
    tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('Close:Q', format=',.2f'),
             alt.Tooltip('RSI:Q', format=',.2f')]
).properties(
    title='RSI',
    width=800,
    height=300)

Oversold = alt.Chart(df).mark_line(color= 'red').encode(
    x='Date:T',
    y='Oversold:Q'
)

Overbought = alt.Chart(df).mark_line(color= 'green').encode(
    x='Date:T',
    y='Overbought:Q'
)

RSI_Chart=RSI+Oversold+Overbought
Stock & RSI_Chart

alt.VConcatChart(...)

### Setting up RSI Strategy 

In [835]:
print('Current Start Date :',df.iloc[0]['Date'])

Current Start Date : 2019-04-29 00:00:00


In [836]:
# Set up your initial investment and time below
df = df.loc[df['Date']>='2019-04-15']
df.reset_index(drop=True,inplace=True)
df.head()

,Date,Open,High,Low,Close,RSI,Overbought,Oversold
0,2019-04-29,68.844815,69.458179,68.745887,69.29,59.59,70,30
1,2019-04-30,69.339473,69.616482,68.538149,68.90,55.22,70,30
2,2019-05-01,69.181187,69.181187,67.064103,67.08,40.34,70,30
3,2019-05-02,67.044318,67.618109,66.361712,67.29,42.27,70,30
4,2019-05-03,67.707141,67.895102,67.049261,67.27,42.13,70,30


In [837]:
#initial values
startAmount=1000
startPrice=df.iloc[0]['Close']
shares=round(startAmount/startPrice,3)

In [838]:
# choose your sell and buy RSI threshold, you would sell when RSI hits the sell threhold and rebuy when RSI drops below buy thresold
RSI_Sell=70
RSI_Buy=60

In [839]:
#RSI Dateframe with Initialize Info
df_rsi = pd.DataFrame(columns = ['Date','Price','Shares', 'Amount','RSI','Buy/Sell','Profit/Loss','Profit/Loss %'])
df_rsi = df_rsi.append({'Date' : df.iloc[0]['Date'], 
              'Price' : startPrice,
              'Shares' : shares, 
              'Amount' : startAmount,
              'RSI': df.iloc[0]['RSI'],
              'Buy/Sell': 'Buy',
              'Profit/Loss':0,
              'Profit/Loss %':0},ignore_index=True)

df_longterm = df_rsi

print('Start Price:', '${:,.2f}'.format(startPrice))
print('Start Shares: ', shares)
print('RSI Sell Signal: >=', RSI_Sell)
print('RSI Buy Signal: <', RSI_Buy)
df_rsi.head()

Start Price: $69.29
Start Shares:  14.432
RSI Sell Signal: >= 70
RSI Buy Signal: < 60


C:\Users\nick\AppData\Local\Temp\ipykernel_35216\2588147253.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({'Date' : df.iloc[0]['Date'],


,Date,Price,Shares,Amount,RSI,Buy/Sell,Profit/Loss,Profit/Loss %
0,2019-04-29 00:00:00,69.29,14.432,1000,59.59,Buy,0,0


In [840]:
## buy and sell stock based on RSI value
Sell_Flag=0
principle=0
profit = 0
saveProfitFlag = 1 # 1: does not reinvest profit
for index, row in df.iterrows():
    if index == 0:
        principle = df_rsi.iloc[-1]['Amount']
    
    

    if index!=0 and row['RSI']>=RSI_Sell and Sell_Flag==0: # SELLING 
        Sell_Flag=1
        Sold_Amount=df_rsi.iloc[-1]['Shares']*row['Close']
        profit = profit + round(Sold_Amount-df_rsi.iloc[-1]['Amount'],2)
        df_rsi = df_rsi.append({
              'Date' : row['Date'], 
              'Price' : row['Close'],
              'Shares' : 0, 
              'Amount' : Sold_Amount,
              'RSI': row['RSI'],
              'Buy/Sell': 'Sell',
              'Profit/Loss': round(Sold_Amount-df_rsi.iloc[-1]['Amount'],2),
              'Profit/Loss %': round((Sold_Amount-df_rsi.iloc[-1]['Amount'])/df_rsi.iloc[-1]['Amount']*100,2)                              
                            },ignore_index=True)
    
    elif index!=0 and row['RSI']<RSI_Buy and Sell_Flag==1: # BUYING
        Sell_Flag=0
        if saveProfitFlag == 1 and df_rsi.iloc[-1]['Amount'] > principle:
            df_rsi = df_rsi.append({
                'Date' : row['Date'], 
                'Price' : row['Close'],
                'Shares' : principle/row['Close'], 
                'Amount' : principle, #df_rsi.iloc[-1]['Amount'],
                'RSI': row['RSI'],
                'Buy/Sell': 'Buy',
                'Profit/Loss':0,
                'Profit/Loss %':0                              
                                },ignore_index=True)
        else:
            df_rsi = df_rsi.append({
                'Date' : row['Date'], 
                'Price' : row['Close'],
                'Shares' : df_rsi.iloc[-1]['Amount']/row['Close'], 
                'Amount' : df_rsi.iloc[-1]['Amount'],
                'RSI': row['RSI'],
                'Buy/Sell': 'Buy',
                'Profit/Loss':0,
                'Profit/Loss %':0                              
                                },ignore_index=True)
        
        
# sell on the last day and calculate the final profit/loss        
if df_rsi.iloc[-1]['Buy/Sell']=='Buy':
    Sold_Amount=df.iloc[-1]['Close']*df_rsi.iloc[-1]['Shares']
    profit = profit + Sold_Amount-df_rsi.iloc[-1]['Amount']
    df_rsi = df_rsi.append({
              'Date' : df.iloc[-1]['Date'], 
              'Price' : df.iloc[-1]['Close'],
              'Shares' : 0, 
              'Amount' : Sold_Amount,
              'RSI': df.iloc[-1]['RSI'],
              'Buy/Sell': 'Sell',
              'Profit/Loss':Sold_Amount-df_rsi.iloc[-1]['Amount'],
              'Profit/Loss %':round((Sold_Amount-df_rsi.iloc[-1]['Amount'])/df_rsi.iloc[-1]['Amount']*100,2)                              
                            },ignore_index=True)
    
# add buy and sell threshold
df_rsi['Sell'] = RSI_Sell
df_rsi['Buy'] = RSI_Buy

C:\Users\nick\AppData\Local\Temp\ipykernel_35216\3107239076.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({
C:\Users\nick\AppData\Local\Temp\ipykernel_35216\3107239076.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({
C:\Users\nick\AppData\Local\Temp\ipykernel_35216\3107239076.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({
C:\Users\nick\AppData\Local\Temp\ipykernel_35216\3107239076.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rsi = df_rsi.append({
C:\Users\nick\AppData\Local\Temp\ipykernel_35216\3107239076.py:16: FutureWarning: The frame.

In [841]:
df_rsi

,Date,Price,Shares,Amount,RSI,Buy/Sell,Profit/Loss,Profit/Loss %,Sell,Buy
0,2019-04-29 00:00:00,69.29,14.432,1000,59.59,Buy,0,0,70,60
1,2019-06-18 00:00:00,71.29,0,1028.85728,70.58,Sell,28.86,2.89,70,60
2,2019-06-25 00:00:00,70.34,14.216662,1000,59.74,Buy,0,0,70,60
3,2019-06-28 00:00:00,72.8,0,1034.972988,70.03,Sell,34.97,3.5,70,60
4,2019-08-05 00:00:00,78.73,12.701639,1000,56.42,Buy,0,0,70,60
5,2019-08-08 00:00:00,83.66,0,1062.619078,70.04,Sell,62.62,6.26,70,60
6,2019-08-12 00:00:00,80.65,12.399256,1000,57.36,Buy,0,0,70,60
7,2020-01-07 00:00:00,88.35,0,1095.474272,70.05,Sell,95.47,9.55,70,60
8,2020-01-31 00:00:00,90.98,10.991427,1000,56.16,Buy,0,0,70,60
9,2020-02-13 00:00:00,96.34,0,1058.914047,71.38,Sell,58.91,5.89,70,60


In [842]:
Final_RSI = alt.Chart(df_rsi).mark_circle(color='Blue',size=90).encode(
    x='Date:T',
    y='RSI:Q',
    color='Buy/Sell',
        tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('RSI:Q', format=',.2f')]
).properties(
    width=800,
    height=300
)

Sell_Line = alt.Chart(df_rsi).mark_line(strokeDash=[5,5],color= 'red').encode(
    x='Date:T',
    y='Sell:Q',
    tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('RSI:Q', format=',.2f')]
)

Buy_Line = alt.Chart(df_rsi).mark_line(strokeDash=[5,5], color= 'green').encode(
    x='Date:T',
    y='Buy:Q',
    tooltip=[alt.Tooltip('date:T', format='%m/%d'),
             alt.Tooltip('RSI:Q', format=',.2f')]
)

#RSI+Final_RSI+Sell_Line+Buy_Line

In [843]:
df_longterm = df_rsi.iloc[[0, -1]]
df_longterm.reset_index(drop=True,inplace=True)
df_longterm.at[1, 'Amount'] = df_longterm.iloc[-1]['Price']*df_longterm.iloc[0]['Shares']
df_longterm.at[1, 'Buy/Sell'] = 'Sell'
df_longterm.at[1, 'Profit/Loss'] = df_longterm.iloc[-1]['Amount']-df_longterm.iloc[0]['Amount']
df_longterm.at[1, 'Profit/Loss %'] = round(df_longterm.at[1, 'Profit/Loss']/df_longterm.iloc[0]['Amount']*100,2)

In [844]:
#Stock&(RSI+Final_RSI+Sell_Line+Buy_Line)
print("Profit: "+str(profit))
print("Principle: "+str(principle))
print("Percent gain: "+str(100*profit/principle))

Profit: 487.76857166569835
Principle: 1000
Percent gain: 48.77685716656983


In [845]:
df_rsi.sum(axis = 0, skipna=True)

C:\Users\nick\AppData\Local\Temp\ipykernel_35216\3369551733.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_rsi.sum(axis = 0, skipna=True)


Price                                                      2047.95
Shares                                                  123.663756
Amount                                                22383.729322
RSI                                                        1393.77
Buy/Sell         BuySellBuySellBuySellBuySellBuySellBuySellBuyS...
Profit/Loss                                             487.768572
Profit/Loss %                                                49.28
Sell                                                          1540
Buy                                                           1320
dtype: object

In [846]:
display(df_rsi,df_longterm, titles=['RSI Strategy ','Long Term Buy&Hold'])

,Date,Price,Shares,Amount,RSI,Buy/Sell,Profit/Loss,Profit/Loss %,Sell,Buy
0,2019-04-29 00:00:00,69.29,14.432,1000,59.59,Buy,0,0,70,60
1,2019-06-18 00:00:00,71.29,0,1028.85728,70.58,Sell,28.86,2.89,70,60
2,2019-06-25 00:00:00,70.34,14.216662,1000,59.74,Buy,0,0,70,60
3,2019-06-28 00:00:00,72.8,0,1034.972988,70.03,Sell,34.97,3.5,70,60
4,2019-08-05 00:00:00,78.73,12.701639,1000,56.42,Buy,0,0,70,60
5,2019-08-08 00:00:00,83.66,0,1062.619078,70.04,Sell,62.62,6.26,70,60
6,2019-08-12 00:00:00,80.65,12.399256,1000,57.36,Buy,0,0,70,60
7,2020-01-07 00:00:00,88.35,0,1095.474272,70.05,Sell,95.47,9.55,70,60
8,2020-01-31 00:00:00,90.98,10.991427,1000,56.16,Buy,0,0,70,60
9,2020-02-13 00:00:00,96.34,0,1058.914047,71.38,Sell,58.91,5.89,70,60
